In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [2]:
pd.options.display.float_format = '{:.3f}'.format

In [3]:
def read_from_datasource(filename):
    df = pd.read_parquet(filename)
    
    duration = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = duration
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    # df.duration.describe(percentiles=[0.95, 0.98, 0.99])

    filter_duration = (df.duration > 1) & (df.duration < 60)
    df = df[filter_duration]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    # sns.displot(df.duration)
    return df


In [4]:
    
def create_train_data(df_train, df_val):
    
    # categorical = ["PU_DO"]
    categorical = ["PULocationID", "DOLocationID"]
    numerical = ["trip_distance"]

    dv = DictVectorizer()

    train_dicts = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)

    val_dicts = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dicts)

    return (X_train, X_val)

## HomeWork

Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [ ]:
df_home_train = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
total_coumns = len(df_home_train.columns)
print(f"Total number of columns are {total_coumns}")

Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [ ]:
duration = df_home_train['tpep_dropoff_datetime'] - df_home_train['tpep_pickup_datetime']
df_home_train['duration'] = duration

standard_deviation_of_trip_duration = df_home_train['duration'].std()
print(f"The Standard Devation of trip duration in January 2023 is {standard_deviation_of_trip_duration}")

Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [ ]:
original_records = len(df_home_train)

df_home_train.duration = df_home_train.duration.apply(lambda td: td.total_seconds()/60)
filter_duration = (df_home_train.duration > 1) & (df_home_train.duration < 60)
df_home_train = df_home_train[filter_duration]

new_records = len(df_home_train)
fraction_remaining = new_records / original_records

print(f"Remaining fraction of records after dropping outliers are {fraction_remaining * 100}")

Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [ ]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]
df_home_train[categorical] = df_home_train[categorical].astype(str)

dv_home = DictVectorizer()

train_home_dict = df_home_train[categorical + numerical].to_dict(orient='records')
                                                       
X_home_train = dv_home.fit_transform(train_home_dict)

print(f"The Dimensionality of the matrix (number of columns) is {X_home_train.shape[1]}")


Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [ ]:

y_home_train = df_home_train["duration"].values

lin_regression = LinearRegression()
lin_regression.fit(X_home_train, y_home_train)

y_home_predict = lin_regression.predict(X_home_train)

rmse_home = root_mean_squared_error(y_home_train, y_home_predict)
print(f'Root Mean Squared Error is {rmse_home}')

Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [15]:
df_home_val = read_from_datasource("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

X_home_train, X_home_val = create_train_data(df_home_train, df_home_val)

In [16]:
target_home = "duration"
y_home_train = df_home_train["duration"].values

y_home_val = df_home_val["duration"].values

In [ ]:
y_home_pred = lin_regression.predict(X_home_val)

rmse_val = root_mean_squared_error(y_home_val, y_home_pred)
print(f'Root Mean Squared Error is {rmse_val}')